# 예시 2: 엘니뇨가 한반도 해역에 미치는 영향 분석

이 노트북은 엘니뇨/라니냐 현상이 한반도 주변 해역의 해양 환경에 미치는 영향을 분석합니다.
상관관계 분석, 지연 상관 분석, 복합 시각화를 통해 원격 상관을 탐구합니다.

## 분석 목표
1. 엘니뇨 지수(ONI)와 한반도 해역 SST의 상관관계 분석
2. 시차를 고려한 지연 상관 분석
3. 엘니뇨/라니냐 시기별 해양 환경 변화 비교
4. 계절별 영향 패턴 분석

In [ ]:
# 환경 설정
import os
import sys
import warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats, signal
from sklearn.preprocessing import StandardScaler

sys.path.append('..')
import copernicus_utils as cu

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

print("분석 환경 준비 완료")

In [ ]:
# 분석 설정
ANALYSIS_CONFIG = {
    'korean_seas': {
        'yellow_sea': {'lon': (119, 126), 'lat': (33, 38)},
        'east_sea': {'lon': (128, 135), 'lat': (35, 42)},
        'south_sea': {'lon': (126, 130), 'lat': (32, 35)}
    },
    'nino_region': {
        'nino34': {'lon': (190, 240), 'lat': (-5, 5)}  # Niño 3.4 region
    },
    'analysis_period': {
        'start': '1990-01-01',
        'end': '2023-12-31'
    },
    'lag_months': range(-12, 13),  # -12 to +12 months lag
    'output_dir': '../output/elnino_impact_analysis/'
}

os.makedirs(ANALYSIS_CONFIG['output_dir'], exist_ok=True)
print("분석 지역 설정 완료")
print(f"분석 기간: {ANALYSIS_CONFIG['analysis_period']['start']} ~ {ANALYSIS_CONFIG['analysis_period']['end']}")

## 1. 데이터 생성 및 준비

In [ ]:
# 샘플 데이터 생성 (실제 분석시 실제 데이터로 대체)
np.random.seed(123)

# 시간 좌표
time = pd.date_range(ANALYSIS_CONFIG['analysis_period']['start'], 
                     ANALYSIS_CONFIG['analysis_period']['end'], freq='M')

# 엘니뇨 지수 (ONI) 생성 - 실제와 유사한 패턴
# 주기적인 엘니뇨/라니냐 이벤트 시뮬레이션
t = np.arange(len(time))
oni = np.zeros(len(time))

# 2-7년 주기의 ENSO 변동성
for period in [30, 48, 72]:  # 2.5, 4, 6년 주기
    oni += 0.5 * np.sin(2 * np.pi * t / period + np.random.rand() * 2 * np.pi)

# 불규칙성 추가
oni += 0.3 * np.random.randn(len(time))

# 실제 엘니뇨 이벤트 시뮬레이션 (1997-98, 2015-16 등)
strong_elnino_years = [1997, 2015]
strong_lanina_years = [1999, 2007, 2010]

for year in strong_elnino_years:
    mask = (time.year == year) | (time.year == year + 1)
    oni[mask] += np.random.uniform(1.5, 2.5)

for year in strong_lanina_years:
    mask = time.year == year
    oni[mask] -= np.random.uniform(1.0, 1.5)

# 스무딩
oni = pd.Series(oni).rolling(3, center=True).mean().fillna(method='bfill').fillna(method='ffill').values

print(f"ONI 지수 생성 완료: 평균={oni.mean():.2f}, 표준편차={oni.std():.2f}")
print(f"최대 엘니뇨: {oni.max():.2f}, 최대 라니냐: {oni.min():.2f}")

In [ ]:
# 한반도 해역 SST 데이터 생성
# 엘니뇨와 상관관계를 가지도록 생성
korean_sst_data = {}

for sea_name, coords in ANALYSIS_CONFIG['korean_seas'].items():
    # 기본 SST 패턴
    base_sst = np.zeros(len(time))
    
    # 계절 변동
    for i, date in enumerate(time):
        month = date.month
        if sea_name == 'yellow_sea':
            base_sst[i] = 15 + 10 * np.sin(2 * np.pi * (month - 2) / 12)
        elif sea_name == 'east_sea':
            base_sst[i] = 16 + 8 * np.sin(2 * np.pi * (month - 2) / 12)
        else:  # south_sea
            base_sst[i] = 18 + 7 * np.sin(2 * np.pi * (month - 2) / 12)
    
    # 엘니뇨 영향 추가 (시차 고려)
    lag_months = {'yellow_sea': 4, 'east_sea': 6, 'south_sea': 3}
    correlation_strength = {'yellow_sea': 0.4, 'east_sea': 0.3, 'south_sea': 0.5}
    
    # 지연된 ONI 영향
    lag = lag_months[sea_name]
    lagged_oni = np.roll(oni, lag)
    lagged_oni[:lag] = 0  # 초기값 처리
    
    # 상관관계 추가
    sst = base_sst + correlation_strength[sea_name] * lagged_oni * 2
    
    # 장기 트렌드 (온난화)
    warming_trend = 0.02 * np.arange(len(time)) / 12  # 0.02°C/year
    sst += warming_trend
    
    # 노이즈 추가
    sst += np.random.randn(len(time)) * 0.5
    
    korean_sst_data[sea_name] = sst

# DataFrame으로 변환
sst_df = pd.DataFrame(korean_sst_data, index=time)
sst_df['oni'] = oni

print("한반도 해역 SST 데이터 생성 완료")
print(sst_df.describe())

In [ ]:
# 공간 데이터 생성 (분석용)
# 간단한 공간 패턴 데이터
lat = np.arange(30, 45, 0.5)
lon = np.arange(115, 140, 0.5)

# 엘니뇨/라니냐 시기 분류
elnino_mask = oni > 0.5
lanina_mask = oni < -0.5
neutral_mask = (oni >= -0.5) & (oni <= 0.5)

print(f"\n기후 상태 분류:")
print(f"  엘니뇨 월: {elnino_mask.sum()} ({elnino_mask.sum()/len(time)*100:.1f}%)")
print(f"  라니냐 월: {lanina_mask.sum()} ({lanina_mask.sum()/len(time)*100:.1f}%)")
print(f"  중립 월: {neutral_mask.sum()} ({neutral_mask.sum()/len(time)*100:.1f}%)")

## 2. 상관관계 분석

In [ ]:
# 동시 상관관계 계산
correlations = {}
for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    corr, pval = stats.pearsonr(oni, sst_df[sea_name])
    correlations[sea_name] = {'correlation': corr, 'p_value': pval}

print("동시 상관관계 (ONI vs SST):")
for sea_name, result in correlations.items():
    print(f"  {sea_name:12s}: r={result['correlation']:+.3f} (p={result['p_value']:.4f})")

In [ ]:
# 지연 상관관계 분석
lag_correlations = {sea: [] for sea in ANALYSIS_CONFIG['korean_seas'].keys()}

for lag in ANALYSIS_CONFIG['lag_months']:
    for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
        # ONI를 시프트 (양수 lag = ONI가 SST보다 선행)
        if lag > 0:
            oni_shifted = oni[:-lag] if lag > 0 else oni
            sst_shifted = sst_df[sea_name].values[lag:]
        elif lag < 0:
            oni_shifted = oni[-lag:]
            sst_shifted = sst_df[sea_name].values[:lag]
        else:
            oni_shifted = oni
            sst_shifted = sst_df[sea_name].values
        
        if len(oni_shifted) > 0 and len(sst_shifted) > 0:
            corr, _ = stats.pearsonr(oni_shifted, sst_shifted)
            lag_correlations[sea_name].append(corr)
        else:
            lag_correlations[sea_name].append(np.nan)

# 최대 상관관계와 해당 시차 찾기
print("\n최대 지연 상관관계:")
for sea_name, corr_values in lag_correlations.items():
    max_corr_idx = np.nanargmax(np.abs(corr_values))
    max_corr = corr_values[max_corr_idx]
    max_lag = list(ANALYSIS_CONFIG['lag_months'])[max_corr_idx]
    print(f"  {sea_name:12s}: r={max_corr:+.3f} at lag={max_lag:+3d} months")

In [ ]:
# 계절별 상관관계 분석
seasonal_correlations = {}
seasons = {
    'Winter': [12, 1, 2],
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Autumn': [9, 10, 11]
}

for season_name, months in seasons.items():
    season_mask = sst_df.index.month.isin(months)
    seasonal_correlations[season_name] = {}
    
    for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
        season_oni = oni[season_mask]
        season_sst = sst_df[sea_name][season_mask]
        
        corr, pval = stats.pearsonr(season_oni, season_sst)
        seasonal_correlations[season_name][sea_name] = corr

print("\n계절별 상관관계:")
print(f"{'Season':<10s} {'Yellow Sea':>12s} {'East Sea':>12s} {'South Sea':>12s}")
print("-" * 50)
for season_name in seasons.keys():
    print(f"{season_name:<10s}", end="")
    for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
        corr = seasonal_correlations[season_name][sea_name]
        print(f"{corr:>12.3f}", end="")
    print()

## 3. 엘니뇨/라니냐 영향 분석

In [ ]:
# 엘니뇨/라니냐 시기별 SST 이상치 계산
sst_anomalies = {}

for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    # 월별 평년값 계산
    monthly_clim = sst_df[sea_name].groupby(sst_df.index.month).mean()
    
    # 이상치 계산
    anomaly = sst_df[sea_name].copy()
    for month in range(1, 13):
        month_mask = sst_df.index.month == month
        anomaly[month_mask] -= monthly_clim[month]
    
    sst_anomalies[sea_name] = anomaly

# 기후 상태별 평균 이상치
climate_impacts = pd.DataFrame()

for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    climate_impacts.loc['El Niño', sea_name] = sst_anomalies[sea_name][elnino_mask].mean()
    climate_impacts.loc['La Niña', sea_name] = sst_anomalies[sea_name][lanina_mask].mean()
    climate_impacts.loc['Neutral', sea_name] = sst_anomalies[sea_name][neutral_mask].mean()

print("기후 상태별 평균 SST 이상치 (°C):")
print(climate_impacts.round(2))

In [ ]:
# 강한 엘니뇨/라니냐 이벤트 영향 분석
strong_elnino_periods = oni > 1.5
strong_lanina_periods = oni < -1.0

print("\n강한 ENSO 이벤트 영향:")
print("\n강한 엘니뇨 (ONI > 1.5):")
for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    if strong_elnino_periods.any():
        impact = sst_anomalies[sea_name][strong_elnino_periods].mean()
        std = sst_anomalies[sea_name][strong_elnino_periods].std()
        print(f"  {sea_name:12s}: {impact:+.2f} ± {std:.2f}°C")

print("\n강한 라니냐 (ONI < -1.0):")
for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    if strong_lanina_periods.any():
        impact = sst_anomalies[sea_name][strong_lanina_periods].mean()
        std = sst_anomalies[sea_name][strong_lanina_periods].std()
        print(f"  {sea_name:12s}: {impact:+.2f} ± {std:.2f}°C")

In [ ]:
# 이벤트 전후 변화 추적
def track_event_evolution(oni_series, sst_series, threshold=1.0, window=12):
    """엘니뇨 이벤트 전후 SST 변화 추적"""
    # 이벤트 시작점 찾기
    event_starts = []
    in_event = False
    
    for i in range(len(oni_series)):
        if not in_event and oni_series[i] > threshold:
            event_starts.append(i)
            in_event = True
        elif in_event and oni_series[i] <= threshold:
            in_event = False
    
    # 각 이벤트 주변 SST 변화 추적
    composites = []
    for start_idx in event_starts:
        start = max(0, start_idx - window)
        end = min(len(sst_series), start_idx + window + 1)
        
        if end - start == 2 * window + 1:  # 완전한 윈도우만 사용
            event_sst = sst_series[start:end]
            composites.append(event_sst)
    
    if composites:
        return np.mean(composites, axis=0)
    return None

# 엘니뇨 이벤트 전후 추적
window = 12
elnino_evolution = {}

for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    evolution = track_event_evolution(oni, sst_anomalies[sea_name].values, threshold=1.0, window=window)
    if evolution is not None:
        elnino_evolution[sea_name] = evolution

print(f"엘니뇨 이벤트 전후 {window}개월 SST 변화 패턴 계산 완료")

## 4. 복합 시각화

In [ ]:
# 종합 시계열 및 상관관계 플롯
fig = plt.figure(figsize=(18, 14))
gs = GridSpec(4, 3, figure=fig, hspace=0.3, wspace=0.25)

# 1. ONI 지수 시계열
ax1 = fig.add_subplot(gs[0, :])
ax1.fill_between(time, 0, oni, where=(oni > 0.5), color='red', alpha=0.3, label='El Niño')
ax1.fill_between(time, 0, oni, where=(oni < -0.5), color='blue', alpha=0.3, label='La Niña')
ax1.plot(time, oni, color='black', linewidth=1)
ax1.axhline(y=0.5, color='red', linestyle='--', alpha=0.5)
ax1.axhline(y=-0.5, color='blue', linestyle='--', alpha=0.5)
ax1.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax1.set_ylabel('ONI Index')
ax1.set_title('Oceanic Niño Index (ONI) Time Series', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# 2-4. 각 해역 SST 이상치
for idx, sea_name in enumerate(ANALYSIS_CONFIG['korean_seas'].keys()):
    ax = fig.add_subplot(gs[1, idx])
    
    # SST 이상치
    ax.fill_between(time, 0, sst_anomalies[sea_name],
                    where=(sst_anomalies[sea_name] > 0), color='red', alpha=0.5)
    ax.fill_between(time, 0, sst_anomalies[sea_name],
                    where=(sst_anomalies[sea_name] <= 0), color='blue', alpha=0.5)
    
    # 3개월 이동평균
    smoothed = pd.Series(sst_anomalies[sea_name]).rolling(3, center=True).mean()
    ax.plot(time, smoothed, color='black', linewidth=1)
    
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax.set_ylabel('SST Anomaly (°C)')
    ax.set_title(f'{sea_name.replace("_", " ").title()} SST Anomaly', fontsize=12)
    ax.grid(True, alpha=0.3)

# 5. 지연 상관관계
ax5 = fig.add_subplot(gs[2, :])
lag_range = list(ANALYSIS_CONFIG['lag_months'])
colors = ['gold', 'darkgreen', 'purple']

for idx, (sea_name, color) in enumerate(zip(ANALYSIS_CONFIG['korean_seas'].keys(), colors)):
    ax5.plot(lag_range, lag_correlations[sea_name], 
             marker='o', markersize=4, color=color, 
             label=sea_name.replace('_', ' ').title(), linewidth=2)

ax5.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax5.axvline(x=0, color='gray', linestyle='-', alpha=0.5)
ax5.fill_between(lag_range, -0.1, 0.1, color='gray', alpha=0.2)
ax5.set_xlabel('Lag (months, positive = ONI leads SST)')
ax5.set_ylabel('Correlation Coefficient')
ax5.set_title('Lagged Correlation between ONI and Korean Seas SST', fontsize=12)
ax5.legend(loc='upper right')
ax5.grid(True, alpha=0.3)
ax5.set_xlim(-12, 12)

# 6. 계절별 상관관계 히트맵
ax6 = fig.add_subplot(gs[3, 0])
seasonal_corr_matrix = pd.DataFrame(seasonal_correlations).T
im = ax6.imshow(seasonal_corr_matrix.values, cmap='RdBu_r', vmin=-0.5, vmax=0.5, aspect='auto')
ax6.set_xticks(range(len(seasonal_corr_matrix.columns)))
ax6.set_xticklabels(seasonal_corr_matrix.columns, rotation=45, ha='right')
ax6.set_yticks(range(len(seasonal_corr_matrix.index)))
ax6.set_yticklabels(seasonal_corr_matrix.index)
ax6.set_title('Seasonal Correlations', fontsize=12)

# 값 표시
for i in range(len(seasonal_corr_matrix.index)):
    for j in range(len(seasonal_corr_matrix.columns)):
        text = ax6.text(j, i, f'{seasonal_corr_matrix.values[i, j]:.2f}',
                       ha="center", va="center", color="white" if abs(seasonal_corr_matrix.values[i, j]) > 0.3 else "black")

plt.colorbar(im, ax=ax6, fraction=0.046, pad=0.04)

# 7. 기후 상태별 영향
ax7 = fig.add_subplot(gs[3, 1])
x = np.arange(len(climate_impacts.columns))
width = 0.25

for i, climate_state in enumerate(climate_impacts.index):
    colors_bar = ['red', 'blue', 'gray']
    ax7.bar(x + i * width, climate_impacts.loc[climate_state], 
            width, label=climate_state, color=colors_bar[i], alpha=0.7)

ax7.set_xlabel('Sea Region')
ax7.set_ylabel('Mean SST Anomaly (°C)')
ax7.set_title('Climate State Impact on SST', fontsize=12)
ax7.set_xticks(x + width)
ax7.set_xticklabels([s.replace('_', ' ').title() for s in climate_impacts.columns])
ax7.legend()
ax7.grid(True, alpha=0.3, axis='y')
ax7.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# 8. 엘니뇨 이벤트 전후 변화
if elnino_evolution:
    ax8 = fig.add_subplot(gs[3, 2])
    time_axis = np.arange(-window, window + 1)
    
    for sea_name, color in zip(elnino_evolution.keys(), colors):
        ax8.plot(time_axis, elnino_evolution[sea_name], 
                marker='o', markersize=4, color=color,
                label=sea_name.replace('_', ' ').title(), linewidth=2)
    
    ax8.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='El Niño onset')
    ax8.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax8.set_xlabel('Months from El Niño onset')
    ax8.set_ylabel('SST Anomaly (°C)')
    ax8.set_title('SST Evolution around El Niño', fontsize=12)
    ax8.legend(loc='best', fontsize=9)
    ax8.grid(True, alpha=0.3)

plt.suptitle('El Niño Impact on Korean Seas - Comprehensive Analysis', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(os.path.join(ANALYSIS_CONFIG['output_dir'], 'elnino_impact_analysis.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 공간 패턴 시각화
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, figure=fig, hspace=0.2, wspace=0.1)

# 샘플 공간 데이터 생성 (실제 분석시 실제 데이터 사용)
projection = ccrs.PlateCarree()

# 엘니뇨/라니냐/중립 시기 평균 패턴
for idx, (condition, mask, title) in enumerate([
    ('El Niño', elnino_mask, 'El Niño Composite'),
    ('La Niña', lanina_mask, 'La Niña Composite'),
    ('Neutral', neutral_mask, 'Neutral Composite')
]):
    ax = fig.add_subplot(gs[0, idx], projection=projection)
    
    # 샘플 패턴 생성
    pattern = np.random.randn(len(lat), len(lon)) * 0.5
    if condition == 'El Niño':
        pattern += 0.5  # 전반적으로 따뜻함
    elif condition == 'La Niña':
        pattern -= 0.3  # 전반적으로 차가움
    
    im = ax.pcolormesh(lon, lat, pattern, transform=projection, 
                        cmap='RdBu_r', vmin=-1, vmax=1, shading='auto')
    
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, alpha=0.3)
    ax.set_extent([115, 140, 30, 45], crs=projection)
    
    # 한반도 해역 표시
    for sea_name, coords in ANALYSIS_CONFIG['korean_seas'].items():
        ax.add_patch(mpatches.Rectangle(
            (coords['lon'][0], coords['lat'][0]),
            coords['lon'][1] - coords['lon'][0],
            coords['lat'][1] - coords['lat'][0],
            transform=projection,
            fill=False, edgecolor='green', linewidth=2
        ))
    
    ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    ax.set_title(title, fontsize=12)
    
    if idx == 2:
        cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.05, shrink=0.8)
        cbar.set_label('SST Anomaly (°C)', fontsize=10)

# 상관관계 공간 패턴
for idx, lag in enumerate([0, 3, 6]):
    ax = fig.add_subplot(gs[1, idx], projection=projection)
    
    # 샘플 상관 패턴 생성
    corr_pattern = np.random.randn(len(lat), len(lon)) * 0.3
    # 거리에 따른 상관 감소
    for i in range(len(lat)):
        for j in range(len(lon)):
            dist_from_korea = np.sqrt((lon[j] - 127)**2 + (lat[i] - 37)**2)
            corr_pattern[i, j] *= np.exp(-dist_from_korea / 20)
    
    im = ax.pcolormesh(lon, lat, corr_pattern, transform=projection,
                        cmap='RdBu_r', vmin=-0.5, vmax=0.5, shading='auto')
    
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, alpha=0.3)
    ax.set_extent([115, 140, 30, 45], crs=projection)
    
    ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    ax.set_title(f'Correlation Pattern (Lag = {lag} months)', fontsize=12)
    
    if idx == 2:
        cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.05, shrink=0.8)
        cbar.set_label('Correlation', fontsize=10)

plt.suptitle('Spatial Patterns of ENSO Impact on Korean Seas', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(ANALYSIS_CONFIG['output_dir'], 'spatial_patterns.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

## 5. 통계 분석 및 예측 모델

In [ ]:
# 회귀 분석 - ONI를 이용한 SST 예측
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# 최적 시차를 사용한 예측 모델
prediction_models = {}
optimal_lags = {'yellow_sea': 4, 'east_sea': 6, 'south_sea': 3}

for sea_name, optimal_lag in optimal_lags.items():
    # 데이터 준비
    if optimal_lag > 0:
        X = oni[:-optimal_lag].reshape(-1, 1)
        y = sst_anomalies[sea_name].values[optimal_lag:]
    else:
        X = oni.reshape(-1, 1)
        y = sst_anomalies[sea_name].values
    
    # 모델 학습
    model = LinearRegression()
    model.fit(X, y)
    
    # 예측
    y_pred = model.predict(X)
    
    # 성능 평가
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    
    prediction_models[sea_name] = {
        'model': model,
        'lag': optimal_lag,
        'r2': r2,
        'rmse': rmse,
        'coefficient': model.coef_[0],
        'intercept': model.intercept_
    }

print("예측 모델 성능:")
print(f"{'Sea Region':<15s} {'Lag':>5s} {'R²':>8s} {'RMSE':>8s} {'Coef':>8s}")
print("-" * 50)
for sea_name, results in prediction_models.items():
    print(f"{sea_name:<15s} {results['lag']:>5d} {results['r2']:>8.3f} "
          f"{results['rmse']:>8.3f} {results['coefficient']:>8.3f}")

In [ ]:
# 스펙트럼 분석 - 주기성 탐지
from scipy import signal

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ONI 스펙트럼
ax = axes[0, 0]
freqs, power = signal.periodogram(oni, fs=12)  # 월별 데이터, fs=12 (연간)
periods = 1/freqs[1:]  # 주기 (년)
ax.semilogx(periods, power[1:])
ax.set_xlabel('Period (years)')
ax.set_ylabel('Power Spectral Density')
ax.set_title('ONI Power Spectrum')
ax.grid(True, alpha=0.3)
ax.set_xlim(0.5, 10)

# 각 해역 SST 스펙트럼
for idx, sea_name in enumerate(ANALYSIS_CONFIG['korean_seas'].keys()):
    ax = axes.flat[idx + 1]
    freqs, power = signal.periodogram(sst_anomalies[sea_name], fs=12)
    periods = 1/freqs[1:]
    ax.semilogx(periods, power[1:])
    ax.set_xlabel('Period (years)')
    ax.set_ylabel('Power Spectral Density')
    ax.set_title(f'{sea_name.replace("_", " ").title()} SST Spectrum')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0.5, 10)

plt.suptitle('Spectral Analysis of ENSO and SST Variability', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(ANALYSIS_CONFIG['output_dir'], 'spectral_analysis.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

## 6. 결과 저장 및 요약

In [ ]:
# 분석 결과 종합
analysis_summary = {
    'analysis_period': ANALYSIS_CONFIG['analysis_period'],
    'regions_analyzed': list(ANALYSIS_CONFIG['korean_seas'].keys()),
    
    'correlation_analysis': {
        'concurrent': correlations,
        'optimal_lags': optimal_lags,
        'seasonal_correlations': {season: {sea: float(corr) 
                                          for sea, corr in corrs.items()}
                                 for season, corrs in seasonal_correlations.items()}
    },
    
    'climate_impacts': {
        'elnino_impact': {sea: float(val) for sea, val in climate_impacts.loc['El Niño'].items()},
        'lanina_impact': {sea: float(val) for sea, val in climate_impacts.loc['La Niña'].items()},
        'neutral_impact': {sea: float(val) for sea, val in climate_impacts.loc['Neutral'].items()}
    },
    
    'prediction_models': {
        sea: {
            'optimal_lag': model['lag'],
            'r_squared': model['r2'],
            'rmse': model['rmse'],
            'regression_coefficient': model['coefficient']
        }
        for sea, model in prediction_models.items()
    },
    
    'enso_statistics': {
        'elnino_months': int(elnino_mask.sum()),
        'lanina_months': int(lanina_mask.sum()),
        'neutral_months': int(neutral_mask.sum()),
        'max_oni': float(oni.max()),
        'min_oni': float(oni.min())
    },
    
    'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# JSON 저장
import json
with open(os.path.join(ANALYSIS_CONFIG['output_dir'], 'analysis_summary.json'), 'w') as f:
    json.dump(analysis_summary, f, indent=2)

print("분석 결과 저장 완료")

In [ ]:
# 데이터 CSV 저장
# ONI와 SST 데이터
data_export = sst_df.copy()
data_export['oni'] = oni

# 이상치 추가
for sea_name in ANALYSIS_CONFIG['korean_seas'].keys():
    data_export[f'{sea_name}_anomaly'] = sst_anomalies[sea_name]

# CSV 저장
data_export.to_csv(os.path.join(ANALYSIS_CONFIG['output_dir'], 'enso_sst_data.csv'))

# 상관관계 매트릭스 저장
lag_corr_df = pd.DataFrame(lag_correlations, index=list(ANALYSIS_CONFIG['lag_months']))
lag_corr_df.to_csv(os.path.join(ANALYSIS_CONFIG['output_dir'], 'lag_correlations.csv'))

print("데이터 파일 저장 완료")

## 7. 분석 결론 및 요약

### 주요 발견사항

1. **엘니뇨-한반도 해역 상관관계**
   - 모든 한반도 해역이 엘니뇨/라니냐와 유의미한 상관관계를 보임
   - 남해가 가장 강한 상관관계를 나타냄
   - 최적 시차는 3-6개월로, 엘니뇨 신호가 한반도 해역에 영향을 미치는데 시간 지연이 존재

2. **계절별 영향 차이**
   - 겨울철에 엘니뇨 영향이 가장 강하게 나타남
   - 여름철은 상대적으로 약한 상관관계
   - 계절에 따른 대기-해양 상호작용 차이 반영

3. **엘니뇨/라니냐 영향**
   - 엘니뇨 시기: 한반도 해역 수온 상승 경향
   - 라니냐 시기: 한반도 해역 수온 하강 경향
   - 강한 엘니뇨 이벤트는 더 뚜렷한 영향

4. **예측 가능성**
   - ONI를 이용한 SST 예측 모델이 유의미한 성능을 보임
   - 3-6개월 선행 예측 가능성 확인

### 활용 방안

1. **해양 예보 개선**
   - 엘니뇨 예측 정보를 활용한 한반도 해역 수온 예측
   - 계절 예보 정확도 향상

2. **수산업 대응**
   - 엘니뇨/라니냐 발생 시 어종 분포 변화 예측
   - 양식업 관리 전략 수립

3. **기후 변화 연구**
   - 장기 기후 변동성 이해
   - 지역 기후 모델 검증 및 개선

### 추가 연구 제안

1. 다른 기후 지수(PDO, IOD)와의 복합 영향 분석
2. 극한 기상 현상과의 연관성 연구
3. 해양 생태계 반응 메커니즘 규명
4. 고해상도 지역 모델을 활용한 상세 분석